<a href="https://colab.research.google.com/github/chaos44/generative_deep_learning/blob/main/chap3/3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone -b tensorflow_2 https://github.com/davidADSP/GDL_code.git

Cloning into 'GDL_code'...
remote: Enumerating objects: 391, done.
remote: Total 391 (delta 0), reused 0 (delta 0), pack-reused 391
Receiving objects: 100% (391/391), 22.13 MiB | 35.14 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [23]:
cd GDL_code/

[Errno 2] No such file or directory: 'GDL_code/'
/content/GDL_code/GDL_code


## **Autoencoder**

In [8]:
import os

from utils.loaders import load_mnist
from models.AE import Autoencoder

# **Set parameters**

In [31]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

# **Load the data**

In [32]:
(x_train, y_train), (x_test, y_test) = load_mnist()

# **Define the structure of the neural network**

In [33]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if MODE == 'build':
    AE.save(RUN_FOLDER)
else:
    AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [34]:
AE.encoder.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 7, 7, 64)        

In [35]:
AE.decoder.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 14, 14, 64)      

# **Train the autoencoder**

In [36]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [37]:
AE.compile(LEARNING_RATE)

In [38]:
AE.train(     
    x_train[:1000]
    , batch_size = BATCH_SIZE
    , epochs = 200
    , run_folder = RUN_FOLDER
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
28/32 [=========================>....] - ETA: 0s - loss: 0.1989
Epoch 00001: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 9ms/step - loss: 0.1900
Epoch 2/200
28/32 [=========================>....] - ETA: 0s - loss: 0.1089
Epoch 00002: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 7ms/step - loss: 0.1091
Epoch 3/200
30/32 [===========================>..] - ETA: 0s - loss: 0.1077
Epoch 00003: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 6ms/step - loss: 0.1076
Epoch 4/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0830
Epoch 00004: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 6ms/step - loss: 0.0818
Epoch 5/200
29/32 [==========================>...] - ETA: 0s - loss: 0.0670
Epoch 00005: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [=

# **VAE Analysis**
## **imports**

In [21]:
%load_ext autoreload
%autoreload 2

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist, load_model

In [40]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

## **imports**

In [41]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## **architecture**

In [42]:
vae = load_model(VariationalAutoencoder, RUN_FOLDER)

ValueError: ignored